# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Setting up Apache Cassandra and data modeling. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the project directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Connecting to cluster
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace 
try :
    session.execute("""
                       CREATE KEYSPACE IF NOT EXISTS sparkifydb
                       WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as err :
    print(err)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as err:
    print(err)

### Now we are going to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## We will create queries to answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### For the first question we will use the sessionId as a partition key and itemInSession as a clustering key since we will use them to search for the required data, and we will also include the rest of the required data i.e artist, song title, song length.

In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
session.execute("""CREATE TABLE IF NOT EXISTS song_sessions
                   (
                       session_id int,
                       item_in_session int,
                       artist_name text,
                       song_title text,
                       song_length float,
                       PRIMARY KEY(session_id, item_in_session)
                    )
""")

In [9]:
# Creating variables for column index locations for better typing and readability
artist_name_iloc = 0
first_name_iloc = 1
gender_iloc = 2
item_in_session_iloc = 3
last_name_iloc = 4
song_length_iloc = 5
level_name_iloc = 6
location_name_iloc = 7
session_id_iloc = 8
song_title_iloc = 9
user_id_iloc = 10

In [10]:
file = 'event_datafile_new.csv'

# Opening csv to rean and insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_sessions (session_id, item_in_Session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Inserting the required data from csv line using index location
        session.execute(query,
                             (
                                 int(line[session_id_iloc]),
                                 int(line[item_in_session_iloc]),
                                 line[artist_name_iloc],
                                 line[song_title_iloc],
                                 float(line[song_length_iloc])
                              )
                       )

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Writing SELECT statement to verify the data was entered into the table
try:
    results = session.execute("""SELECT artist_name, song_title, song_length FROM song_sessions WHERE session_id = 338 AND item_in_session = 4""")
except Exception as err:
    print(err)
    
results_df = pd.DataFrame(data = results._current_rows, columns = ['artist_name', 'song_title', 'song_lenght'])
results_df.head()

,artist_name,song_title,song_lenght
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### For the 2nd question we will use the userId and sessionId as a composite partition key for better precision and then use itemInSession as a clustering key since we want to sort the data by it, and we will include the rest of the required data i.e artist, song, first name, last name.

In [12]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
session.execute("""CREATE TABLE IF NOT EXISTS song_user_sessions
                   (
                       user_id int,
                       session_id int,
                       item_in_session int,
                       song_title text,
                       artist_name text,
                       first_name text,
                       last_name text, 
                       PRIMARY KEY((user_id, session_id), item_in_session)
                    )
""")


In [13]:
# Opening csv to rean and insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_sessions (user_id, session_id, item_in_session, song_title, artist_name, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Inserting the required data from csv line using index location
        session.execute(query, 
                            (
                                int(line[user_id_iloc]),
                                int(line[session_id_iloc]),
                                int(line[item_in_session_iloc]),
                                line[song_title_iloc],
                                line[artist_name_iloc],
                                line[first_name_iloc],
                                line[last_name_iloc]
                            )
                       )

In [14]:
# Writing SELECT statement to verify the data was entered into the table
try:
    results = session.execute("""SELECT artist_name, song_title, first_name, last_name FROM song_user_sessions WHERE user_id = 10 AND session_id = 182""")
    
except Exception as err:
    print(err)
    
results_df = pd.DataFrame(data = results._current_rows, columns = ['artist_name', 'song_title', 'first_name', 'last_name'])
results_df.head()

,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### For the 3rd question we will use the song title as a partition key and the userId as a clustering key to create a composite key unique enough to include a single interaction between any user and a given song and also be more efficient by including only one interaction (since any additional sessions with the same song and user will overwrite the prevoius value) since we don't need to know any info about the sessions and we just want the user info.

In [15]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("""CREATE TABLE IF NOT EXISTS song_users
                   (
                        song_title text,
                        user_id int,
                        first_name text,
                        last_name text, 
                        PRIMARY KEY (song_title, user_id)
                    )
""")

In [16]:
# Opening csv to rean and insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        # Inserting the required data from csv line using index location
        session.execute(query,
                             (
                                 line[song_title_iloc],
                                 int(line[user_id_iloc]),
                                 line[first_name_iloc],
                                 line[last_name_iloc]
                             )
                       )

In [17]:
try:
    results = session.execute("""SELECT first_name, last_name FROM song_users WHERE song_title = 'All Hands Against His Own' """)
    
except Exception as err:
    print(err)
    
results_df = pd.DataFrame(data = results._current_rows, columns = ['first_name', 'last_name'])
results_df.head()

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [18]:
## TO-DO: Drop the table before closing out the sessions
session.execute("""DROP TABLE song_sessions""")
session.execute("""DROP TABLE song_user_sessions""")
session.execute("""DROP TABLE song_users""")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()